<a href="https://colab.research.google.com/github/daryoush/colab-julia-1.5/blob/master/ExperimentsWithBoradcastinPythonGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from numpy.random import uniform
import numpy as np
import torch
import timeit
#  https://towardsdatascience.com/speed-up-your-python-code-with-broadcasting-and-pytorch-64fbd31b359

# for every point determine if it is in any of the boxes
def in_boxes(boxes, points):
  # (N, 4, 2) = boxes.shape
  # (M, 2) = points.shape
  w = np.zeros(points.shape[0])
  for (i, point) in enumerate(points):
   in_box = False
   for box in boxes:
     (A, B, C, D) = box
     AP = (point - A)
     AB = (B - A)
     AD = (D - A)
     cond0 = 0 < np.dot(AP, AB) < np.dot(AB, AB)
     cond1 = 0 < np.dot(AP, AD) < np.dot(AD, AD)
     in_box = in_box or (cond0 and cond1)   
     if in_box : 
       break   # don't need to check any more rects.
   if in_box:
     w[i] = 1
   else:
     w[i] = 0
  return w


def in_boxes_bc(boxes, points):
  # (N, 4, 2) = boxes.shape
  # (M, 2) = points.shape
  (A, B, C, D) = np.split(boxes, 4, axis=1)  # (N, 1, 2)
  AM = (points[None, ...] - A)  # (N, M, 2)
  AB = (B - A)  # (N, 1, 2)
  AD = (D - A)  # (N, 1, 2) 
    
  AM_AB = np.sum(AM * AB, axis=-1)  # (N, M)
  AB_AB = np.sum(AB * AB, axis=-1)  # (N, 1)
  AM_AD = np.sum(AM * AD, axis=-1)  # (N, M)
  AD_AD = np.sum(AD * AD, axis=-1)  # (N, 1)
    
  cond0 = (0 < AM_AB) & (AM_AB < AB_AB)  # (N, M)
  cond1 = (0 < AM_AD) & (AM_AD < AD_AD)  # (N, M)
    
  in_box = cond0 & cond1  # (N, M) = in_box.shape
  w = np.any(in_box, axis=0)
  return w






In [10]:

npoints = 1000000
points = uniform(0, 10, size=(npoints,2))   # npoints 2d values 

rects =  np.array([[[0,0], [1,0], [0,1], [1, 1]], [[1,1], [2,0], [3,1], [2,2]],[[5,5], [5,10], [10,5], [10,10]],[[2,2],[4,0],[6,2], [4,4]],[[6,6],[6,8],[10,8],[10,6]]])

points.shape
rects.shape



(5, 4, 2)

In [14]:
timeit.timeit(lambda : in_boxes(rects, points), number=2)


75.07031807499999

In [19]:
timeit.timeit(lambda : in_boxes_bc(rects, points), number=100)


29.64130264299996

In [16]:
def in_boxes_torch(boxes, points):
  boxes = torch.DoubleTensor(boxes).cuda()
  points = torch.DoubleTensor(points).cuda()
    
  dd = torch.chunk(boxes, 4, dim=1)
  (A, B, C, D) = dd
  AM = (points[None, ...] - A)
  AB = (B - A)
  AD = (D - A)
  AM_AB = torch.sum(AM * AB, dim=-1)
  AB_AB = torch.sum(AB * AB, dim=-1)
  AM_AD = torch.sum(AM * AD, dim=-1)
  AD_AD = torch.sum(AD * AD, dim=-1)
  cond0 = (0 < AM_AB) & (AM_AB < AB_AB)
  cond1 = (0 < AM_AD) & (AM_AD < AD_AD)
    
  in_box = cond0 & cond1
  # PyTorch does not have anything corresponding to np.any()
  w = (torch.sum(in_box, dim=0) > 0)
  return w.cpu()


In [18]:
timeit.timeit(lambda : in_boxes_torch(rects, points), number=100)

0.9152755900000216